In [1]:
import fitz  # PyMuPDF
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

def extract_text_from_pdf(file_path):
    # Abre el archivo PDF
    doc = fitz.open(file_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

def preprocess_data(text):
    # Dividir el texto en líneas y limpiar los datos
    lines = text.split('\n')
    lines = [line.strip() for line in lines if line.strip() != '']
    return lines

def feature_extraction(data):
    # Convertir datos a vectores para el análisis de clustering
    vectorizer = TfidfVectorizer(stop_words='english')
    X = vectorizer.fit_transform(data)
    return X, vectorizer

def cluster_data(X, n_clusters=10):
    # Aplicar K-means para agrupar líneas similares
    kmeans = KMeans(n_clusters=n_clusters)
    kmeans.fit(X)
    return kmeans.labels_

def organize_data(lines, labels):
    # Organizar los datos etiquetados en un diccionario
    grouped_data = {}
    for label, line in zip(labels, lines):
        if label in grouped_data:
            grouped_data[label].append(line)
        else:
            grouped_data[label] = [line]
    return grouped_data

def create_json_format(grouped_data):
    # Convertir los datos agrupados en un formato JSON específico
    json_data = {
       "nombre_cliente": r"Titular del contrato: (\w+ [\w ]+)",
        "dni_cliente": r"NIF: (\w+)",
        "calle_cliente": r"Dirección de suministro: ([\w ,]+)",
        "cp_cliente": r"(\d{5})",
        "población_cliente": r"Dirección suministro: .+ (\d{5}) (\w+)",
        "provincia_cliente": r"provincia_cliente: (\w+)",
        "nombre_comercializadora": r"nombre_comercializadora: (\w+)",
        "cif_comercializadora": r"CIF (\w+)",
        "dirección_comercializadora": r"Domicilio Social: ([\w ,]+)",
        "cp_comercializadora": r"Domicilio Social: .+?(\d{5})",
        "población_comercializadora": r"Domicilio Social: .+? \d{5} (\w+)",
        "provincia_comercializadora": r"provincia_comercializadora: (\w+)",
        "número_factura": r"Nº factura: (\w+)",
        "inicio_periodo": r"Periodo de Facturación: del (\d{2}/\d{2}/\d{4})",
        "fin_periodo": r"a (\d{2}/\d{2}/\d{4})",
        "importe_factura": r"TOTAL IMPORTE FACTURA(\d+,\d{2}) €",
        "fecha_cargo": r"Fecha de cargo: (\d{2} de \w+ de \d{4})",
        "consumo_periodo": r"Consumo\n(\d+ kWh)",
        "potencia_contratada": r"Potencia contratada: (\d,\d{3}) kW"
    }
    return json.dumps(json_data, indent=4)

# Uso del código
pdf_text = extract_text_from_pdf("./training/factura_0.pdf") #CAMBIA LA RUTA Y EL NOMBRE DE ARCHIVO
processed_text = preprocess_data(pdf_text)
X, vectorizer = feature_extraction(processed_text)
labels = cluster_data(X)
grouped_data = organize_data(processed_text, labels)
json_output = create_json_format(grouped_data)

print(json_output)


C:\Users\AdmInfo\AppData\Roaming\Python\Python311\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


{
    "nombre_cliente": "Titular del contrato: (\\w+ [\\w ]+)",
    "dni_cliente": "NIF: (\\w+)",
    "calle_cliente": "Direcci\u00f3n de suministro: ([\\w ,]+)",
    "cp_cliente": "(\\d{5})",
    "poblaci\u00f3n_cliente": "Direcci\u00f3n suministro: .+ (\\d{5}) (\\w+)",
    "provincia_cliente": "provincia_cliente: (\\w+)",
    "nombre_comercializadora": "nombre_comercializadora: (\\w+)",
    "cif_comercializadora": "CIF (\\w+)",
    "direcci\u00f3n_comercializadora": "Domicilio Social: ([\\w ,]+)",
    "cp_comercializadora": "Domicilio Social: .+?(\\d{5})",
    "poblaci\u00f3n_comercializadora": "Domicilio Social: .+? \\d{5} (\\w+)",
    "provincia_comercializadora": "provincia_comercializadora: (\\w+)",
    "n\u00famero_factura": "N\u00ba factura: (\\w+)",
    "inicio_periodo": "Periodo de Facturaci\u00f3n: del (\\d{2}/\\d{2}/\\d{4})",
    "fin_periodo": "a (\\d{2}/\\d{2}/\\d{4})",
    "importe_factura": "TOTAL IMPORTE FACTURA(\\d+,\\d{2}) \u20ac",
    "fecha_cargo": "Fecha de cargo:

In [2]:
import fitz  # PyMuPDF
import re
import json

def extract_text_from_pdf(file_path):
    doc = fitz.open(file_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

def extract_fields(text):
    data = {}
    patterns = {
        "nombre_cliente": r"Titular del contrato: (\w+ [\w ]+)",
        "dni_cliente": r"NIF: (\w+)",
        "calle_cliente": r"Dirección de suministro: ([\w ,]+)",
        "cp_cliente": r"(\d{5})",
        "población_cliente": r"Dirección suministro: .+ (\d{5}) (\w+)",
        "provincia_cliente": r"provincia_cliente: (\w+)",
        "nombre_comercializadora": r"nombre_comercializadora: (\w+)",
        "cif_comercializadora": r"CIF (\w+)",
        "dirección_comercializadora": r"Domicilio Social: ([\w ,]+)",
        "cp_comercializadora": r"Domicilio Social: .+?(\d{5})",
        "población_comercializadora": r"Domicilio Social: .+? \d{5} (\w+)",
        "provincia_comercializadora": r"provincia_comercializadora: (\w+)",
        "número_factura": r"Nº factura: (\w+)",
        "inicio_periodo": r"Periodo de Facturación: del (\d{2}/\d{2}/\d{4})",
        "fin_periodo": r"a (\d{2}/\d{2}/\d{4})",
        "importe_factura": r"TOTAL IMPORTE FACTURA(\d+,\d{2}) €",
        "fecha_cargo": r"Fecha de cargo: (\d{2} de \w+ de \d{4})",
        "consumo_periodo": r"Consumo\n(\d+ kWh)",
        "potencia_contratada": r"Potencia contratada: (\d,\d{3}) kW"
    }
    
    for key, pattern in patterns.items():
        match = re.search(pattern, text)
        if match:
            data[key] = match.group(1)
        else:
            data[key] = "No encontrado"
    
    return data

def create_json(data):
    return json.dumps(data, indent=4, ensure_ascii=False)

# Proceso de ejemplo para un archivo PDF
pdf_text = extract_text_from_pdf("./training/factura_0.pdf") #CAMBIA LA RUTA Y EL NOMBRE DE ARCHIVO
extracted_data = extract_fields(pdf_text)
json_output = create_json(extracted_data)

print(json_output)

{
    "nombre_cliente": "Conrado Daniel Iglesias ",
    "dni_cliente": "73635161V",
    "calle_cliente": "Calle la Solana, La Fueva, Huesca ",
    "cp_cliente": "50436",
    "población_cliente": "No encontrado",
    "provincia_cliente": "No encontrado",
    "nombre_comercializadora": "No encontrado",
    "cif_comercializadora": "B90393497",
    "dirección_comercializadora": "CRT",
    "cp_comercializadora": "No encontrado",
    "población_comercializadora": "No encontrado",
    "provincia_comercializadora": "No encontrado",
    "número_factura": "SV5043664894",
    "inicio_periodo": "26/08/2018",
    "fin_periodo": "25/09/2018",
    "importe_factura": "No encontrado",
    "fecha_cargo": "30 de septiembre de 2018",
    "consumo_periodo": "No encontrado",
    "potencia_contratada": "4,799"
}


In [3]:
import fitz  # PyMuPDF
import re
import json

def extract_text_from_pdf(file_path):
    # Abre el archivo PDF y extrae todo el texto
    doc = fitz.open(file_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

def parse_and_create_json(text):
    # Diccionario para almacenar los datos
    json_data = {}
    
    # Expresiones regulares para identificar cada campo relevante
    fields = {
        "nombre_cliente": [r"Titular del contrato: (.+)", r"Nombre: (.+)"],
        "dni_cliente": [r"NIF: (\w+)"],
        "calle_cliente": [r"Dirección de suministro: (.+)", r"Dirección suministro: (.+)"],
        "cp_cliente": [r"Dirección de suministro: .*, (\d{5})"],
        "población_cliente": [r"Dirección de suministro: .*, \d{5} (.+)",r"Dirección de suministro: [^,]*, ([^,]+),"],
        "provincia_cliente": [r"Dirección de suministro: .*, \d{5} .+, (.+)",r"Dirección de suministro: .*, \w+, (.+)"],
        "nombre_comercializadora": [r"comercializadora: (.+),"],
        "cif_comercializadora": [r"CIF (\w+)",],
        "dirección_comercializadora": [r"Domicilio Social: (.+)"],
        "cp_comercializadora": [r"Domicilio Social: .*, (\d{5})"],
        "población_comercializadora": [r"Domicilio Social: .*, \d{5} (.+)"],
        "provincia_comercializadora": [r"Domicilio Social: .*, \d{5} .+, (.+)"],
        "número_factura": [r"Nº factura: (\w+)"],
        "inicio_periodo": [r"Periodo de Facturación: del (\d{2}/\d{2}/\d{4})",r"Período de (\d{2}\.\d{2}\.\d{4})"],
        "fin_periodo": [r"a (\d{2}/\d{2}/\d{4})", r"a (\d{2}\.\d{2}\.\d{4})"],
        "importe_factura": [r"TOTAL IMPORTE FACTURA\s+(\d+,\d{2}) €",r"(\d+,\d{2})\s*€\s*Total a pagar"],
        "fecha_cargo": [r"Fecha de cargo: (\d{2} de \w+ de \d{4})", r"Fecha de cargo: (\d{2}\.\d{2}\.\d{4})"],
        "consumo_periodo": [r"Consumo en el periodo (\d+)",r"en el periodo\s(.+)",r"en el periodo\s+(\d+)\s+kWh"],
        "potencia_contratada": [r"Potencia contratada: (\d,\d{3}) kW",r"Término de potencia \(([\d,]+) kW\)"]
    }
    
    # Extrae cada campo usando su respectiva expresión regular
    for field, patterns in fields.items():
        found = False
        if not isinstance(patterns, list):
            patterns = [patterns]  # Convierte en lista si no es una lista
        for pattern in patterns:
            match = re.search(pattern, text, re.MULTILINE)
            if match:
                json_data[field] = match.group(1)
                found = True
                break  # Salir del bucle si se encuentra una coincidencia
        if not found:
            json_data[field] = "No encontrado"

    return json.dumps(json_data, indent=4, ensure_ascii=False)

# Uso del código
file_path = "./training/factura_0.pdf" #CAMBIA LA RUTA Y EL NOMBRE DE ARCHIVO
pdf_text = extract_text_from_pdf(file_path)
json_output = parse_and_create_json(pdf_text)

print(json_output)

{
    "nombre_cliente": "Conrado Daniel Iglesias ",
    "dni_cliente": "73635161V",
    "calle_cliente": "Calle la Solana, La Fueva, Huesca ",
    "cp_cliente": "No encontrado",
    "población_cliente": "La Fueva",
    "provincia_cliente": "No encontrado",
    "nombre_comercializadora": "No encontrado",
    "cif_comercializadora": "B90393497",
    "dirección_comercializadora": "CRT.SEVILLA-MADRID, KM 524, CAMINO DE LA PASTORA ",
    "cp_comercializadora": "No encontrado",
    "población_comercializadora": "No encontrado",
    "provincia_comercializadora": "No encontrado",
    "número_factura": "SV5043664894",
    "inicio_periodo": "26/08/2018",
    "fin_periodo": "25/09/2018",
    "importe_factura": "19,78",
    "fecha_cargo": "30 de septiembre de 2018",
    "consumo_periodo": "0",
    "potencia_contratada": "4,799"
}


In [63]:
#TODOS LOS ARCHIVOS EN JSON

import fitz  # PyMuPDF
import re
import json
import os

def extract_text_from_pdf(file_path):
    doc = fitz.open(file_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

def parse_and_create_json(text):
    json_data = {}
    fields = {
        "nombre_cliente": [r"Titular del contrato: (.+)", r"Nombre: (.+)"],
        "dni_cliente": [r"NIF: (\w+)"],
        "calle_cliente": [r"Dirección de suministro: (.+)", r"Dirección suministro: (.+)"],
        "cp_cliente": [r"Dirección de suministro: .*, (\d{5})"],
        "población_cliente": [r"Dirección de suministro: .*, \d{5} (.+)",r"Dirección de suministro: [^,]*, ([^,]+),"],
        "provincia_cliente": [r"Dirección de suministro: .*, \d{5} .+, (.+)",r"Dirección de suministro: .*, \w+, (.+)"],
        "nombre_comercializadora": [r"comercializadora: (.+),"],
        "cif_comercializadora": [r"CIF (\w+)",],
        "dirección_comercializadora": [r"Domicilio Social: (.+)"],
        "cp_comercializadora": [r"Domicilio Social: .*, (\d{5})"],
        "población_comercializadora": [r"Domicilio Social: .*, \d{5} (.+)"],
        "provincia_comercializadora": [r"Domicilio Social: .*, \d{5} .+, (.+)"],
        "número_factura": [r"Nº factura: (\w+)"],
        "inicio_periodo": [r"Periodo de Facturación: del (\d{2}/\d{2}/\d{4})",r"Período de (\d{2}\.\d{2}\.\d{4})"],
        "fin_periodo": [r"a (\d{2}/\d{2}/\d{4})", r"a (\d{2}\.\d{2}\.\d{4})"],
        "importe_factura": [r"TOTAL IMPORTE FACTURA\s+(\d+,\d{2}) €",r"(\d+,\d{2})\s*€\s*Total a pagar"],
        "fecha_cargo": [r"Fecha de cargo: (\d{2} de \w+ de \d{4})", r"Fecha de cargo: (\d{2}\.\d{2}\.\d{4})"],
        "consumo_periodo": [r"Consumo en el periodo (\d+)",r"en el periodo\s(.+)",r"en el periodo\s+(\d+)\s+kWh"],
        "potencia_contratada": [r"Potencia contratada: (\d,\d{3}) kW",r"Término de potencia \(([\d,]+) kW\)"]
    }
    for field, patterns in fields.items():
        found = False
        for pattern in patterns:
            match = re.search(pattern, text, re.MULTILINE)
            if match:
                json_data[field] = match.group(1)
                found = True
                break
        if not found:
            json_data[field] = "No encontrado"
    return json_data

def process_all_pdfs(directory):
    all_data = []
    for filename in os.listdir(directory):
        if filename.endswith(".pdf"):
            file_path = os.path.join(directory, filename)
            pdf_text = extract_text_from_pdf(file_path)
            json_data = parse_and_create_json(pdf_text)
            all_data.append(json_data)
    return json.dumps(all_data, indent=4, ensure_ascii=False)

# Uso del código
directory = "./training"
json_output = process_all_pdfs(directory)
print(json_output)

[
    {
        "nombre_cliente": "Conrado Daniel Iglesias ",
        "dni_cliente": "73635161V",
        "calle_cliente": "Calle la Solana, La Fueva, Huesca ",
        "cp_cliente": "No encontrado",
        "población_cliente": "La Fueva",
        "provincia_cliente": "No encontrado",
        "nombre_comercializadora": "No encontrado",
        "cif_comercializadora": "B90393497",
        "dirección_comercializadora": "CRT.SEVILLA-MADRID, KM 524, CAMINO DE LA PASTORA ",
        "cp_comercializadora": "No encontrado",
        "población_comercializadora": "No encontrado",
        "provincia_comercializadora": "No encontrado",
        "número_factura": "SV5043664894",
        "inicio_periodo": "26/08/2018",
        "fin_periodo": "25/09/2018",
        "importe_factura": "19,78",
        "fecha_cargo": "30 de septiembre de 2018",
        "consumo_periodo": "0",
        "potencia_contratada": "4,799"
    },
    {
        "nombre_cliente": "LEONARDA JARAMILLO BÁEZ ",
        "dni_cliente": 

In [69]:
#TODOS LOS ARCHIVOS EN UN CSV PARA VER QUÉ HAY QUE CAMBIAR

import fitz  # PyMuPDF
import re
import csv
import os

def extract_text_from_pdf(file_path):
    doc = fitz.open(file_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

def parse_and_create_data(text, filename):
    data = {"nombre_archivo": filename}  # Incluir el nombre del archivo en el diccionario de datos
    fields = {
        "nombre_cliente": [r"Titular del contrato: (.+)", r"Nombre: (.+)"],
        "dni_cliente": [r"NIF: (\w+)"],
        "calle_cliente": [r"Dirección de suministro: (.+)", r"Dirección suministro: (.+)"],
        "cp_cliente": [r"Dirección de suministro: .*, (\d{5})"],
        "población_cliente": [r"Dirección de suministro: .*, \d{5} (.+)", r"Dirección de suministro: [^,]*, ([^,]+),"],
        "provincia_cliente": [r"Dirección de suministro: .*, \d{5} .+, (.+)", r"Dirección de suministro: .*, \w+, (.+)"],
        "nombre_comercializadora": [r"comercializadora: (.+),"],
        "cif_comercializadora": [r"CIF (\w+)"],
        "dirección_comercializadora": [r"Domicilio Social: (.+)"],
        "cp_comercializadora": [r"Domicilio Social: .*, (\d{5})"],
        "población_comercializadora": [r"Domicilio Social: .*, \d{5} (.+)"],
        "provincia_comercializadora": [r"Domicilio Social: .*, \d{5} .+, (.+)"],
        "número_factura": [r"Nº factura: (\w+)"],
        "inicio_periodo": [r"Periodo de Facturación: del (\d{2}/\d{2}/\d{4})", r"Período de (\d{2}\.\d{2}\.\d{4})"],
        "fin_periodo": [r"a (\d{2}/\d{2}/\d{4})", r"a (\d{2}\.\d{2}\.\d{4})"],
        "importe_factura": [r"TOTAL IMPORTE FACTURA\s+(\d+,\d{2}) €", r"(\d+,\d{2})\s*€\s*Total a pagar"],
        "fecha_cargo": [r"Fecha de cargo: (\d{2} de \w+ de \d{4})", r"Fecha de cargo: (\d{2}\.\d{2}\.\d{4})"],
        "consumo_periodo": [r"Consumo en el periodo (\d+)", r"en el periodo\s(.+)", r"en el periodo\s+(\d+)\s+kWh"],
        "potencia_contratada": [r"Potencia contratada: (\d,\d{3}) kW", r"Término de potencia \(([\d,]+) kW\)"]
    }
    for field, patterns in fields.items():
        found = False
        for pattern in patterns:
            match = re.search(pattern, text, re.MULTILINE)
            if match:
                data[field] = match.group(1)
                found = True
                break
        if not found:
            data[field] = "No encontrado"
    return data

def process_all_pdfs(directory, output_csv_file):
    all_data = []
    header = list(parse_and_create_data("", "").keys())  # Extrae las claves para el encabezado CSV, añadiendo el nombre del archivo
    for filename in os.listdir(directory):
        if filename.endswith(".pdf"):
            file_path = os.path.join(directory, filename)
            pdf_text = extract_text_from_pdf(file_path)
            data = parse_and_create_data(pdf_text, filename)
            csv_filename = os.path.splitext(filename)[0] + '.csv'
            csv_path = os.path.join(directory, csv_filename)
            with open(csv_path, 'w', newline='', encoding='utf-8') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=header)
                writer.writeheader()
                writer.writerow(data)
            all_data.append(data)
    
    # Escribir todos los datos en un único archivo CSV
    with open(output_csv_file, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=header)
        writer.writeheader()
        for data in all_data:
            writer.writerow(data)

# Uso del código
directory = "./training/"
output_csv_file = "./output/facturas_output_nou.csv"
process_all_pdfs(directory, output_csv_file)
